In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, utils
from torch.utils import data

import matplotlib.pyplot as plt
import numpy as np

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

In [4]:
transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5,), (0.5,))
                   ])

<h1>dataset 

In [5]:
trainset = datasets.MNIST(
    root      = '../data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.MNIST(
    root      = '../data/', 
    train     = False,
    download  = True,
    transform = transform
)

C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
train_loader = data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE, shuffle=True
)
test_loader = data.DataLoader(
    dataset     = testset,
    batch_size  = BATCH_SIZE, shuffle=True
)

In [7]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

In [8]:
images[0].shape

torch.Size([1, 28, 28])

In [9]:
print(labels)

tensor([6, 1, 1, 5, 4, 0, 3, 6, 1, 6, 5, 6, 3, 4, 6, 4, 7, 0, 2, 8, 0, 9, 9, 6,
        8, 0, 5, 4, 5, 9, 9, 0, 9, 1, 2, 6, 1, 7, 8, 1, 8, 6, 5, 4, 1, 9, 7, 6,
        0, 1, 6, 7, 6, 1, 6, 9, 2, 0, 3, 7, 8, 3, 7, 4])


In [10]:
CLASSES = {
    0: 'zero',
    1: 'one',
    2: 'two',
    3: 'three',
    4: 'four',
    5: 'five',
    6: 'six',
    7: 'seven',
    8: 'eight',
    9: 'nine'
}


for label in labels:
    index = label.item()
    print(CLASSES[index])

six
one
one
five
four
zero
three
six
one
six
five
six
three
four
six
four
seven
zero
two
eight
zero
nine
nine
six
eight
zero
five
four
five
nine
nine
zero
nine
one
two
six
one
seven
eight
one
eight
six
five
four
one
nine
seven
six
zero
one
six
seven
six
one
six
nine
two
zero
three
seven
eight
three
seven
four


In [ ]:
idx = 1

item_img = images[idx]
item_npimg = item_img.squeeze().numpy()
plt.title(CLASSES[labels[idx].item()])
print(item_npimg.shape)
plt.imshow(item_npimg, cmap='gray')
plt.show()

<h1>Linear model

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)  
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [13]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [14]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [15]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.323732
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.311864
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.272814
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.269629
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.260193
[1] Test Loss: 2.2467, Accuracy: 32.41%
Train Epoch: 2 [0/60000 (0%)]	Loss: 2.261455
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.232894
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.148907
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.034325
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.862490
[2] Test Loss: 1.7541, Accuracy: 56.66%
Train Epoch: 3 [0/60000 (0%)]	Loss: 1.801105
Train Epoch: 3 [12800/60000 (21%)]	Loss: 1.555452
Train Epoch: 3 [25600/60000 (43%)]	Loss: 1.526296
Train Epoch: 3 [38400/60000 (64%)]	Loss: 1.255500
Train Epoch: 3 [51200/60000 (85%)]	Loss: 1.244716
[3] Test Loss: 1.0972, Accuracy: 68.34%
Train Epoch: 4 [0/60000 (0%)]	Loss: 1.054251
Train Epoch: 4 [12800/60000 (21%)]	Loss: 1.053030
Train Epoch: 4 [25600/60000 (43%)]	Loss: 1.137899


Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.269801
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.216933
[29] Test Loss: 0.3359, Accuracy: 89.29%
Train Epoch: 30 [0/60000 (0%)]	Loss: 0.392475
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.371376
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.263359
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.244516
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.262344
[30] Test Loss: 0.3275, Accuracy: 89.76%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.325312
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.342637
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.216260
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.306035
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.285057
[31] Test Loss: 0.3224, Accuracy: 89.96%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.421398
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.416357
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.275496
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.336517
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.429863
[32] Test Loss: 0.3156, A

In [ ]:
columns = 6
rows = 6
fig = plt.figure(figsize=(10,10))
 
model.eval()
for i in range(1, columns*rows+1):
    data_idx = np.random.randint(len(testset))
    input_img = testset[data_idx][0].unsqueeze(dim=0).to(DEVICE) 
 
    output = model(input_img)
    _, argmax = torch.max(output, 1)
    pred = CLASSES[argmax.item()]
    label = CLASSES[testset[data_idx][1]]
    
    fig.add_subplot(rows, columns, i)
    if pred == label:
        plt.title(pred + ', right')
        cmap = 'Blues'
    else:
        plt.title('N ' + pred + ' B ' +  label)
        cmap = 'Reds'
    plot_img = testset[data_idx][0][0,:,:]
    plt.imshow(plot_img, cmap=cmap)
    plt.axis('off')
    
plt.show() 